In [1]:
# import a tool that allows you to create different kinds of TESS observations
from SPyFFI import Observation


An instance of ds9 was found to be running before we could
start the 'xpans' name server. You will need to perform a
bit of manual intervention in order to connect this
existing ds9 to Python.

For ds9 version 5.7 and beyond, simply register the
existing ds9 with the xpans name server by selecting the
ds9 File->XPA->Connect menu option. Your ds9 will now be
fully accessible to pyds9 (e.g., it appear in the list
returned by the ds9_targets() routine).

For ds9 versions prior to 5.7, you cannot (easily) register
with xpans, but you can view ds9's File->XPA Information
menu option and pass the value associated with XPA_METHOD
directly to the Python DS9() constructor, e.g.:

    d = DS9('a000101:12345')

The good news is that new instances of ds9 will be
registered with xpans, and will be known to ds9_targets()
and the DS9() constructor.



In [2]:
inputs = dict(

# these are necessary to specify
label='laptopsatdawn_rotated',                # a label that will go into the output directory name
ra = 270.000, dec=66.56071,           # coordinates of the focal plane center
jitter=True,                          # should the spacecraft jitter?

# what outputs should be produced?
todo={1800:1},                       # this will expose 100 images each at 120s cadences

# these options are optional, and can make for interesting movies
jitterscale=1.0,                      # by what factor should we magnify the jitter?
warpspaceandtime=1.0,                 # by what factor do we multiply the speed of light (to see DVA)
counterstep=1,                        # how many cadences (of the specified cadence) do we skip per frame? (to speed up time)

# these options relate to how stars are populated with light curves
fmax=0.00,                            # what fraction of eligible stars should get light curves
magmax=None,                          # stars fainter than this magnitude don't get light curves
extreme=False                        # if set to true, varaibility will be extreme (for movies)
)


In [3]:
# create a FFI of a particular patch of the sky, using catalog stars
obs = Observation.SkyFFI(**inputs)

       [skyffi] Creating a new Observation.
       [skyffi] Setting up the camera for this Observation.
       [camera] Turning on a new TESS camera object.
       [camera] Setting cadence to 2 seconds = 1.0 reads.
       [camera] Pointing the camera at (ra,dec) = 270.000000,66.560710
          [psf] Initializing the TESS point spread function painter.
          [psf] loaded PSF subpixel array definitions from
                ... /Users/zkbt/Cosmos/Data/TESS/FFIs/intermediates/original/pixelarrays.npy
      [perfect] Creating an intrapixel sensitivity map called perfectpixels
       [jitter] Populating the jitterball for 2 second cadence.
        [ucac4] loading a catalog of stars from
                ... /Users/zkbt/Cosmos/Data/TESS/FFIs/intermediates/UCAC4_270.0_66.56071_17.70346373
                ... 75.npy
        [ucac4] found 1420240 stars with 2.23799991608 < V < 17.5189990997


In [4]:
# add light curves to this observation's stellar catalog
obs.addLCs(**inputs)

        [ucac4] 1420240 stars are brighter than 18.258999059; populating 0.0% of them with light
                ... curves


In [5]:
# figure out the corners of the CCD
carto = obs.camera.cartographer
carto.setCCD(obs.camera.ccds[0])
carto.pithy = True

# the corner closest to the 
def report(x,y):
    c = carto.point(x,y,'ccdxy')
    print "{0} pixels".format(c.ccdxy)
    print "{0} pixels".format(c.focalxy)
    print "{0} degrees".format(c.celestial)



print "\nclosest to camera center"
report(0,0)

print "\nfarthest from camera center"
report(2047,2047)

print "\nthe other two corners"
report(0,2047)
report(2047,0)




closest to camera center
ccd1 (x, y) = (0,0) pixels
focalplane (x, y) = (67.1666666667,67.1666666667) pixels
celestial (ra, dec) = (268.994488427,66.9511964863) degrees

farthest from camera center
ccd1 (x, y) = (2047,2047) pixels
focalplane (x, y) = (2114.16666667,2114.16666667) pixels
celestial (ra, dec) = (222.669197832,73.6638867596) degrees

the other two corners
ccd1 (x, y) = (0,2047) pixels
focalplane (x, y) = (67.1666666667,2114.16666667) pixels
celestial (ra, dec) = (268.026038279,78.7576281934) degrees
ccd1 (x, y) = (2047,0) pixels
focalplane (x, y) = (2114.16666667,67.1666666667) pixels
celestial (ra, dec) = (241.081462018,64.0794447339) degrees


In [6]:
# this is PT's outermost corner
ZKBT = carto.point(2047, 2047, 'ccdxy')
print ZKBT.focalrtheta
PT = carto.point(271.8645990732, 83.1011559323114, 'celestial')
print PT.focalrtheta

import numpy as np
#rotation = (PT.focalrtheta.theta - ZKBT.focalrtheta.theta)*180/np.pi
rotation = 45.7863590193


# this rotates the WCS to align the two orientations
obs.camera.wcs.wcs.crota = -np.array([rotation, rotation])

# now, PT's coordinates should appear at the top right of my ccd
print "PT's coordinates are at {0} on my CCD.".format(carto.point(271.8645990732, 83.1011559323114, 'celestial').ccdxy)


focalplane (r, theta) = (2989.88317312,0.785398163397)
focalplane (r, theta) = (2904.11265835,1.58452088078)
PT's coordinates are at ccd1 (x, y) = (1986.35108739,1986.35108738) on my CCD.


In [7]:
# create the observation, generating all exposures
obs.camera.ccds = [obs.camera.ccds[0]]
obs.create(**inputs)

       [camera] Setting cadence to 1800 seconds = 900.0 reads.
       [jitter] Populating the jitterball for 1800 second cadence.
       [jitter] Jittering the camera to 0.0174121180164,0.0102261562642,-0.0302088720421 away from
                ... nominal pointing.
          [ccd] Adding stars.
          [ccd] projecting the starmap onto CCD
          [ccd] setting up this CCD's starmap
          [ccd] the camera already had a catalog of 1420240 elements defined; using it!
          [ccd] taking an intial snapshot at 2457827.0 = 2017.20054757
        [ucac4] projecting catalog 17.201 years relative to 2000
          [ccd]   done!
      [trimmed] projecting catalog 17.201 years relative to 2000
      [trimmed] save projected star catalog /Users/zkbt/Cosmos/Data/TESS/FFIs/outputs/18h00m00s+66d3
                ... 3m39s_laptopsatdawn_rotated/1800s/ccd1/catalog_18h00m00s+66d33m39s_ccd1.txt
          [ccd] taking a snapshot at 2457827.0 = 2017.20054757
      [trimmed] projecting catalog 1

In [8]:
c = obs.camera.ccds[0]
c.expose?

<bound method CCD.expose of <SPyFFI.CCD.CCD object at 0x10f804c50>>